In [1]:
import numpy as np
import pandas as pd

from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score, confusion_matrix
from sklearn.preprocessing import label_binarize

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset

from art.attacks.evasion import SimBA, SpatialTransformation, DeepFool, BasicIterativeMethod, FastGradientMethod, ProjectedGradientDescent
from art.estimators.classification import PyTorchClassifier

import time

/opt/conda/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
head = {
            "model" : '',
            "attack_model": '',
            'epsilon': '',
            'Accuracy': '',
            'Macro Precision': '',
            'Weighted Precision': '',
            'Macro Recall': '',
            'Weighted Recall': '',
            'Macro F1': '',
            'Weighted F1': '',
            # 'Macro AUC': '',
            # 'Weighted AUC': '',
            'TPR': '',
            'FNR': '',
            'TNR': '',
            'FPR': '',
        }
head = pd.DataFrame([head])
head.to_csv("/home/jovyan/UNSW/transfer_attack/attackmodel.csv", mode='a', index=False)




In [3]:
def calculate_performance_metrics(X_test, y_true, model, model_name, attack_name, eps):
    model.eval()
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    
    all_preds = []
    all_labels = []
    probabilities = []

    num_classes = len(np.unique(y_true))
    
    X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
    y_test_tensor = torch.tensor(y_test, dtype=torch.long)
    test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
    test_loader = DataLoader(dataset=test_dataset)

    with torch.no_grad():
        
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            preds = torch.argmax(outputs, dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            probabilities.extend(torch.nn.functional.softmax(outputs, dim=1).cpu().numpy())
        
        all_preds = np.array(all_preds)
        all_labels = np.array(all_labels)
        probabilities = np.array(probabilities)
        
        accuracy = accuracy_score(all_labels, all_preds)

        precision_macro, recall_macro, f1_macro, _ = precision_recall_fscore_support(all_labels, all_preds, average='macro')
        precision_weighted, recall_weighted, f1_weighted, _ = precision_recall_fscore_support(all_labels, all_preds, average='weighted')
    
        # macro_auc = roc_auc_score(label_binarize(all_labels, classes=range(num_classes)), probabilities[:,1], average='macro')
        # weighted_auc = roc_auc_score(label_binarize(all_labels, classes=range(num_classes)), probabilities[:,1], average='weighted')

        cm = confusion_matrix(all_labels, all_preds)

        def calculate_class_metrics_macro(cm, class_index):
            TP = cm[class_index, class_index]
            FP = cm[:, class_index].sum() - TP
            FN = cm[class_index, :].sum() - TP
            TN = cm.sum() - (TP + FP + FN)
            
            TPR = TP / (TP + FN) if (TP + FN) != 0 else 0  
            TNR = TN / (TN + FP) if (TN + FP) != 0 else 0  
            FPR = FP / (FP + TN) if (FP + TN) != 0 else 0  
            FNR = FN / (FN + TP) if (FN + TP) != 0 else 0  
            
            return TPR, TNR, FPR, FNR
            
        metrics = np.array([calculate_class_metrics_macro(cm, i) for i in range(num_classes)])
        TPR_macro, TNR_macro, FPR_macro, FNR_macro = np.mean(metrics, axis=0)

        print(f"Accuracy: {accuracy}")
        
        print("\nmacro")
        print(f"Precision: {precision_macro}\nRecall: {recall_macro}\nF1 Score: {f1_macro}")
    
        print("\nweighted")
        print(f"Precision: {precision_weighted}\nRecall: {recall_weighted}\nF1 Score: {f1_weighted}")
        print()
        
        print(f"Mean FNR: {FNR_macro}\nMean TNR: {TNR_macro}\nMean FPR: {FPR_macro}\nMean TPR: {TPR_macro}")

        new_row = {
            "model" : model_name,
            "attack_model" : attack_name,
            'epsilon': eps,
            'Accuracy': accuracy,
            'Macro Precision': precision_macro,
            'Weighted Precision': precision_weighted,
            'Macro Recall': recall_macro,
            'Weighted Recall': recall_weighted,
            'Macro F1': f1_macro,
            'Weighted F1': f1_weighted,
            # 'Macro AUC': macro_auc,
            # 'Weighted AUC': weighted_auc,
            'TPR': TPR_macro,
            'FNR': FNR_macro,
            'TNR': TNR_macro,
            'FPR': FPR_macro,
        }
        new_row_df = pd.DataFrame([new_row])
        new_row_df.to_csv("/home/jovyan/UNSW/transfer_attack/attackmodel.csv", mode='a', index=False, header=False)

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {device} device")

Using cuda device


In [2]:
x_train = np.load("/home/jovyan/UNSW/x_train.npy")
y_train = np.load("/home/jovyan/UNSW/y_train.npy")
x_val = np.load("/home/jovyan/UNSW/x_val.npy")
y_val = np.load("/home/jovyan/UNSW/y_val.npy")
x_test = np.load("/home/jovyan/UNSW/x_test.npy")
y_test = np.load("/home/jovyan/UNSW/y_test.npy")

In [6]:
input_shape = x_train.shape[1]
output_shape = len(np.unique(y_train))

In [7]:
class DNNModel(nn.Module):
    def __init__(self, input_size, output_size):
        super(DNNModel, self).__init__()
        self.fc1 = nn.Linear(input_size, 50)
        self.fc2 = nn.Linear(50, 30)
        self.fc3 = nn.Linear(30, 20)
        self.fc4 = nn.Linear(20, output_size)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return x

In [8]:
model = DNNModel(input_size=input_shape, output_size=output_shape).to(device)
model.load_state_dict(torch.load("/home/jovyan/UNSW/transfer_attack/dnn_pytorch.pt"))

<All keys matched successfully>

In [9]:
x_train = x_train.astype(np.float32)
x_test = x_test.astype(np.float32)

In [10]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
    
classifier = PyTorchClassifier(
    model=model,
    clip_values=(-5, 5),
    loss=criterion,
    optimizer=optimizer,
    input_shape=(input_shape,),
    nb_classes=output_shape,
    device_type='gpu'
)

In [11]:
from art.attacks.evasion import SimBA, SpatialTransformation, DeepFool, BasicIterativeMethod, FastGradientMethod, ProjectedGradientDescent


In [13]:
epsilon_values = [0.01, 0.1, 0.2, 0.3]

# Iterate over epsilon values
for epsilon in epsilon_values:
    adv_crafter = BasicIterativeMethod(classifier, eps=epsilon, max_iter = 10, verbose=False) # init eps=0.3
    x_test_adv = adv_crafter.generate(x=x_test, y=y_test)

    filename = f'/home/jovyan/UNSW/transfer_attack/x_test_adv_BIM_eps_{epsilon}.npy'
    np.save(filename, x_test_adv)

In [14]:
for epsilon in epsilon_values:
    adv_crafter = FastGradientMethod(classifier, eps=epsilon) # init eps=0.3
    x_test_adv = adv_crafter.generate(x=x_test, y=y_test)

    filename = f'/home/jovyan/UNSW/transfer_attack/x_test_adv_FGSM_eps_{epsilon}.npy'
    np.save(filename, x_test_adv)

In [15]:
for epsilon in epsilon_values:
    adv_crafter = ProjectedGradientDescent(classifier, eps=epsilon, verbose=False, max_iter=10) # init eps=0.3
    x_test_adv = adv_crafter.generate(x=x_test, y=y_test)

    filename = f'/home/jovyan/UNSW/transfer_attack/x_test_adv_PGD_eps_{epsilon}.npy'
    np.save(filename, x_test_adv)

In [ ]:
# DF_eps = [1e-07, 1e-06, 1e-05, 1e-04]
for epsilon in epsilon_values:
    adv_crafter = DeepFool(classifier, epsilon=epsilon, verbose=False, max_iter=10) # init eps=1e-06
    x_test_adv = adv_crafter.generate(x=x_test, y=y_test)

    filename = f'/home/jovyan/UNSW/transfer_attack/x_test_adv_DF_eps_{epsilon}.npy'
    np.save(filename, x_test_adv)


In [ ]:
from art.attacks.evasion import AutoProjectedGradientDescent

for epsilon in epsilon_values:
    adv_crafter = AutoProjectedGradientDescent(classifier, eps=epsilon, max_iter = 10, batch_size=128, verbose = False) # init eps=0.3
    x_test_adv = adv_crafter.generate(x=x_test, y=y_test)

    filename = f'/home/jovyan/UNSW/transfer_attack/x_test_adv_AutoPGD_eps_{epsilon}.npy'
    np.save(filename, x_test_adv)

In [17]:
from art.attacks.evasion import AutoProjectedGradientDescent

for epsilon in epsilon_values:
    adv_crafter = AutoProjectedGradientDescent(classifier, eps=epsilon, max_iter = 10, batch_size=128, verbose = False) # init eps=0.3
    x_test_adv = adv_crafter.generate(x=x_test, y=y_test)

    filename = f'/home/jovyan/UNSW/transfer_attack/x_test_adv_AutoPGD_eps_{epsilon}.npy'
    np.save(filename, x_test_adv)

In [21]:
x_test.dtype

dtype('float32')

In [23]:
y_test = y_test.astype(np.float32)
y_test.dtype


dtype('float32')

In [ ]:
from art.attacks.evasion import BoundaryAttack

epsilon_values = [0.01, 0.1, 0.2, 0.3]

# Iterate over epsilon values
for epsilon in epsilon_values:
    adv_crafter = BoundaryAttack(classifier, epsilon=epsilon, max_iter = 5000, batch_size=128, targeted=False, verbose=False) # init eps=0.3
    x_test_adv = adv_crafter.generate(x=x_test, y=y_test)

    filename = f'/home/jovyan/UNSW/transfer_attack/x_test_adv_BA_eps_{epsilon}.npy'
    np.save(filename, x_test_adv)

In [16]:
epsilon_values = [0.01, 0.1, 0.2, 0.3]

# Iterate over epsilon values
for epsilon in epsilon_values:
    filename = f'/home/jovyan/UNSW/transfer_attack/x_test_adv_BIM_eps_{epsilon}.npy'
    x_test_adv = np.load(filename)

    calculate_performance_metrics(x_test_adv, y_test, model, 'DNN', 'BIM', epsilon)

for epsilon in epsilon_values:
    filename = f'/home/jovyan/UNSW/transfer_attack/x_test_adv_FGSM_eps_{epsilon}.npy'
    x_test_adv = np.load(filename)

    calculate_performance_metrics(x_test_adv, y_test, model, 'DNN', 'FGSM', epsilon)

for epsilon in epsilon_values:
    filename = f'/home/jovyan/UNSW/transfer_attack/x_test_adv_PGD_eps_{epsilon}.npy'
    x_test_adv = np.load(filename)

    calculate_performance_metrics(x_test_adv, y_test, model, 'DNN', 'PGD', epsilon)

# # DF_eps = [1e-07, 1e-06, 1e-05, 1e-04]
# for epsilon in epsilon_values:
#     filename = f'/home/jovyan/UNSW/transfer_attack/x_test_adv_DF_eps_{epsilon}.npy'
#     x_test_adv = np.load(filename)

#     calculate_performance_metrics(x_test_adv, y_test, model, 'DNN', 'DF', epsilon)

Accuracy: 0.8854357687481903

macro
Precision: 0.876212899139358
Recall: 0.8556184012066366
F1 Score: 0.8646819154737567

weighted
Precision: 0.884107669388339
Recall: 0.8854357687481903
F1 Score: 0.8838255479772675

Mean FNR: 0.14438159879336349
Mean TNR: 0.8556184012066366
Mean FPR: 0.14438159879336349
Mean TPR: 0.8556184012066366
Accuracy: 0.5536583867826026

macro
Precision: 0.528526268267531
Recall: 0.5321427512490862
F1 Score: 0.5234451561462188

weighted
Precision: 0.5923069778393741
Recall: 0.5536583867826026
F1 Score: 0.566791461283709

Mean FNR: 0.46785724875091395
Mean TNR: 0.5321427512490862
Mean FPR: 0.46785724875091395
Mean TPR: 0.5321427512490862
Accuracy: 0.052241359644119995

macro
Precision: 0.06917943095793198
Recall: 0.04271562504028567
F1 Score: 0.050582490899224564

weighted
Precision: 0.09121771222269683
Recall: 0.052241359644119995
F1 Score: 0.06491861287573379

Mean FNR: 0.9572843749597143
Mean TNR: 0.04271562504028567
Mean FPR: 0.9572843749597143
Mean TPR: 0.0

In [18]:
for epsilon in epsilon_values:
    filename = f'/home/jovyan/UNSW/transfer_attack/x_test_adv_AutoPGD_eps_{epsilon}.npy'
    x_test_adv = np.load(filename)

    calculate_performance_metrics(x_test_adv, y_test, model, 'DNN', 'AutoPGD', epsilon)

Accuracy: 0.8853655754534048

macro
Precision: 0.8761216536902561
Recall: 0.8555449638764119
F1 Score: 0.8646012044386691

weighted
Precision: 0.8840351026934499
Recall: 0.8853655754534048
F1 Score: 0.88375539692495

Mean FNR: 0.14445503612358818
Mean TNR: 0.8555449638764119
Mean FPR: 0.14445503612358818
Mean TPR: 0.8555449638764119
Accuracy: 0.4996007756359074

macro
Precision: 0.4924367431498884
Recall: 0.4913152904159691
F1 Score: 0.4786596071230793

weighted
Precision: 0.5577924669437923
Recall: 0.4996007756359074
F1 Score: 0.5164045002515989

Mean FNR: 0.5086847095840308
Mean TNR: 0.4913152904159691
Mean FPR: 0.5086847095840308
Mean TPR: 0.4913152904159691
Accuracy: 0.019469865141132393

macro
Precision: 0.027233470540432316
Recall: 0.01646103017143741
F1 Score: 0.019270000555163116

weighted
Precision: 0.035663340991538076
Recall: 0.019469865141132393
F1 Score: 0.024327545035151502

Mean FNR: 0.9835389698285626
Mean TNR: 0.01646103017143741
Mean FPR: 0.9835389698285626
Mean TPR: 

In [ ]:
print("start BA")
for epsilon in epsilon_values:
    filename = f'/home/jovyan/UNSW/transfer_attack/x_test_adv_BA_eps_{epsilon}.npy'
    x_test_adv = np.load(filename)

    calculate_performance_metrics(x_test_adv, y_test, model, 'DNN', 'BA', epsilon)